# BERT for obsession with data labelled from the ontology and evaluate the model using manual label

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install contractions

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/areejnasser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/areejnasser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/areejnasser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer

2023-12-05 21:51:33.460658: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Cleaning data

In [5]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['yy'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [ ]:
df = pd.read_csv('/Users/areejnasser/Desktop/Dataset/post-ontology-label.csv')
df


In [6]:
 df = pd.read_csv('/Users/areejnasser/Desktop/Dataset/posts_with_ontology_labe_and_negative.csv')
 df.describe()

,compulsion,obsession
count,9497.000000,9497.000000
mean,0.267032,0.524903
std,0.442432,0.499406
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [7]:
df['post'] = df['post'].fillna('')
df['post'] = df['post'].apply(process_text)
df.head()


,post,prefLabel,compulsion,obsession,expand1
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."


In [8]:
 df1 = pd.read_csv('/Users/areejnasser/Desktop/Dataset/post-manual-label.csv')
 df1.describe()

,obsession_first_checker,compulsion_first_checker,obsession_second_checker,compulsion_second_checker,compulsion_third_checker
count,499.000000,499.000000,499.000000,499.000000,498.000000
mean,0.332665,0.088176,0.322645,0.086172,0.124498
std,0.471641,0.283836,0.467957,0.280900,0.330481
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Select the first 500 rows
df_subset = df1.head(500).copy()

# Clean the 'post' column
df_subset['post'] = df_subset['post'].fillna('')
df_subset['post'] = df_subset['post'].apply(process_text)

# Filter rows where 'obsession_first_checker' has labels 0 or 1
filtered_df = df_subset[df_subset['obsession_first_checker'].isin([0, 1])].copy()

# Save the cleaned dataset to a new CSV file
filtered_df.to_csv('manual_500_post_with_obsession_label.csv', index=False)

# Display the first few rows of the updated DataFrame
print(filtered_df.head())


                                                post  \
0  grew obsessive thought aptly intrusive thought...   
1  thinking much covid part job well right concer...   
2  fed send day checking excessive checking every...   
3  noticed washing hand lately thing triggering n...   
4  sound stupid moment want download window live ...   

                                           prefLabel  obsession_first_checker  \
0  ['site', 'compulsion', 'obsession', 'rash', 'p...                      1.0   
1  ['singing', 'washing hands', 'anxiety', 'anxie...                      1.0   
2                     ['OCD', 'checking', 'Thought']                      1.0   
3     ['washing hands', 'hope', 'depression', 'OCD']                      1.0   
4                                        ['Thought']                      1.0   

   compulsion_first_checker  obsession_second_checker  \
0                       1.0                       1.0   
1                       1.0                       1.0   
2    

In [11]:
len(filtered_df)

499

## Training and Testing

In [10]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(df, test_size=test_split, stratify=df[['obsession']].values, )
test_df,val_df = train_test_split(test_df,test_size=0.5,stratify=test_df[['obsession']].values)
# Splitting the test set further into validation and new test sets.
# val_df = test_df.sample(frac=0.5)
# test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 7597
Number of rows in validation set: 950
Number of rows in test set: 950


In [11]:
train_x,train_y = train_df['post'], train_df[['obsession']]
test_x,test_y = test_df['post'], test_df[['obsession']]
val_x,val_y = val_df['post'], val_df[['obsession']]

In [12]:
# Count the number of labels as 0 and 1 for each group
print("Training set:")
print(train_y['obsession'].value_counts())

print("\nValidation set:")
print(val_y['obsession'].value_counts())

print("\nTest set:")
print(test_y['obsession'].value_counts())

Training set:
1    3988
0    3609
Name: obsession, dtype: int64

Validation set:
1    498
0    452
Name: obsession, dtype: int64

Test set:
1    499
0    451
Name: obsession, dtype: int64


In [13]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score


# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
obsession_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the training and testing texts
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_y, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_y, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_y, dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:

# Compile the model
obsession_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = obsession_model.fit(
    train_encodings,
    train_labels,
    validation_data=(val_encodings, val_labels),
    epochs=2,
    batch_size=32
)


predictions = obsession_model.predict(test_encodings)

predicted_labels = tf.argmax(predictions.logits, axis=1)
accuracy = accuracy_score(test_labels, predicted_labels)
classification_rep = classification_report(test_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)



# Define the directory path in Google Drive
save_directory = '/Users/areejnasser/Desktop/Dataset/bert_obsession_model2'

# Save the model
obsession_model.save(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)







Epoch 1/2
238/238 [==============================] - 7362s 30s/step - loss: 0.2784 - accuracy: 0.8822 - val_loss: 0.1118 - val_accuracy: 0.9632
Epoch 2/2
30/30 [==============================] - 107s 3s/step
Accuracy: 0.9726315789473684
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.96      0.97       451
           1       0.97      0.98      0.97       499

    accuracy                           0.97       950
   macro avg       0.97      0.97      0.97       950
weighted avg       0.97      0.97      0.97       950

INFO:tensorflow:Assets written to: /Users/areejnasser/Desktop/Dataset/bert_obsession_model2/assets


INFO:tensorflow:Assets written to: /Users/areejnasser/Desktop/Dataset/bert_obsession_model2/assets


('/Users/areejnasser/Desktop/Dataset/bert_obsession_model2/tokenizer_config.json',
 '/Users/areejnasser/Desktop/Dataset/bert_obsession_model2/special_tokens_map.json',
 '/Users/areejnasser/Desktop/Dataset/bert_obsession_model2/vocab.txt',
 '/Users/areejnasser/Desktop/Dataset/bert_obsession_model2/added_tokens.json',
 '/Users/areejnasser/Desktop/Dataset/bert_obsession_model2/tokenizer.json')

In [22]:
from transformers import AutoTokenizer
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report

# Load your trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
obsession_model2 = tf.keras.models.load_model('/Users/areejnasser/Desktop/Dataset/bert_obsession_model2')  # Provide the correct path
obsession_model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

# Tokenize and encode the posts
filtered_encodings = tokenizer(filtered_df['post'].tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Make predictions
predictions = obsession_model2(filtered_encodings)  # Use the model directly without the predict method

# Extract logits from the dictionary
logits = predictions['logits']

# Convert logits to predicted labels
predicted_labels = tf.argmax(logits, axis=1)

# Evaluate the model
accuracy = accuracy_score(filtered_labels, predicted_labels)
classification_rep = classification_report(filtered_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)


Accuracy: 0.6492985971943888
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.67      0.72       333
           1       0.48      0.61      0.54       166

    accuracy                           0.65       499
   macro avg       0.63      0.64      0.63       499
weighted avg       0.68      0.65      0.66       499



Compulsion model 

In [23]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(df, test_size=test_split, stratify=df[['compulsion']].values, )
test_df,val_df = train_test_split(test_df,test_size=0.5,stratify=test_df[['compulsion']].values)
# Splitting the test set further into validation and new test sets.
# val_df = test_df.sample(frac=0.5)
# test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 7597
Number of rows in validation set: 950
Number of rows in test set: 950


In [24]:
train_x,train_y = train_df['post'], train_df[['compulsion']]
test_x,test_y = test_df['post'], test_df[['compulsion']]
val_x,val_y = val_df['post'], val_df[['compulsion']]

In [25]:
# Count the number of labels as 0 and 1 for each group
print("Training set:")
print(train_y['compulsion'].value_counts())

print("\nValidation set:")
print(val_y['compulsion'].value_counts())

print("\nTest set:")
print(test_y['compulsion'].value_counts())

Training set:
0    5568
1    2029
Name: compulsion, dtype: int64

Validation set:
0    696
1    254
Name: compulsion, dtype: int64

Test set:
0    697
1    253
Name: compulsion, dtype: int64


In [26]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score


# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
compulsion_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the training and testing texts
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_y, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_y, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_y, dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:

# Compile the model
compulsion_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = compulsion_model.fit(
    train_encodings,
    train_labels,
    validation_data=(val_encodings, val_labels),
    epochs=2,
    batch_size=32
)


predictions = compulsion_model.predict(test_encodings)

predicted_labels = tf.argmax(predictions.logits, axis=1)
accuracy = accuracy_score(test_labels, predicted_labels)
classification_rep = classification_report(test_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)



# Define the directory path in Google Drive
save_directory = '/Users/areejnasser/Desktop/Dataset/bert_compulsion_model2'

# Save the model
obsession_model.save(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)







Epoch 1/2
  3/238 [..............................] - ETA: 46:13 - loss: 0.6403 - accuracy: 0.7292 

In [ ]:
from transformers import AutoTokenizer
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report

# Load your trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
compulsion_model2 = tf.keras.models.load_model('/Users/areejnasser/Desktop/Dataset/bert_compulsion_model2')  # Provide the correct path
compulsion_model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

# Tokenize and encode the posts
filtered_encodings = tokenizer(filtered_df['post'].tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Make predictions
predictions = compulsion_model2(filtered_encodings)  # Use the model directly without the predict method

# Extract logits from the dictionary
logits = predictions['logits']

# Convert logits to predicted labels
predicted_labels = tf.argmax(logits, axis=1)

# Evaluate the model
accuracy = accuracy_score(filtered_labels, predicted_labels)
classification_rep = classification_report(filtered_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)
